In [2]:
import mitsuba as mi
import drjit as dr
mi.set_variant('llvm_ad_rgb')

[mitsuba] Warning: Couldn't import the ipywidgets package. Installing this package is required for the system to properly log messages and print in Jupyter notebooks!


In [3]:
scene = mi.load_file('scenes/cornell-box/cbox.xml', integrator='prb')
params = mi.traverse(scene)
img = mi.render(scene, spp=16)
key = 'RightWallBSDF.brdf_0.reflectance.value'

# Save the original value
param_ref = mi.Color3f(params[key])

# Set another color value and update the scene
params[key] = mi.Color3f(0.01, 0.2, 0.9)
params.update();



RuntimeError: ​[xml.cpp:1413] "scenes/cornell-box/cbox.xml": file does not exist!

In [3]:
opt = mi.ad.Adam(lr=0.05)
opt[key] = params[key]
params.update(opt);

def mse(image):
    return dr.mean(dr.sqr(image - img))


In [4]:
errors = []
iteration_count = 50
for it in range(iteration_count):
    image = mi.render(scene, params, spp=4) # noisy differentiable rendering
    loss = mse(image) # objective function from current rendered image
    dr.backward(loss) # backpropagation through rendering process
    opt.step() # gradient descent step
    opt[key] = dr.clamp(opt[key], 0.0, 1.0) # post-process the optimized parameters to ensure legal color values
    params.update(opt)
    # Track the difference between the current color and the true value
    err_ref = dr.sum(dr.sqr(param_ref - params[key]))
    print(f"Iteration {it:02d}: parameter error = {err_ref[0]:6f}", end='\r')
    errors.append(err_ref)
print('\nOptimization complete.')  

image_final = mi.render(scene, spp=128)
mi.util.convert_to_bitmap(image_final)


Iteration 49: parameter error = 1.868381
Optimization complete.


Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [1024, 1024],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 3 MiB of image data ]
]